# The 1994, M6.7 Northridge earthquake. Including a kinematic, finite earthquake source and topography 

We use SeisSol to combine a kinematic finite earthquake source model with complex topography. Our example is the 1994 Mw 6.7 Northridge earthquake that ruptured a ‘blind’ thrusting, previously unkown fault and was the 
most costly American earthquake since 1906 featuring very strong shaking and ground acceleration.
![](Northridge_DynRup.png)
*Dynamic rupture and wave propagation simulation based on the 1994 Northridge earthquake with 3D velocity model CVM-H and toporgaphy, from [Rettenberger et al., 2016](https://dl.acm.org/doi/10.1145/2938615.2938618)*


## Part 1 - mesh generation using the Gmsh Python API

<img src="Northridge_mesh" width="50%">

We will be introducing an open-source workflow for generating high quality unstructured tetrahedral meshes for SeisSol using gmsh. Gmsh is an automatic three-dimensional finite element mesh generator with
built-in pre- and post-processing facilities. 

We provide the generated mesh in this repository.

The next few cells include the detailed setup of how it was generated - unfold the cells to see all steps. For more information please consult [SeisSol's gmsh documentation](https://seissol.readthedocs.io/en/latest/gmsh.html) and 
+ http://gmsh.info/
+ https://gitlab.onelab.info/gmsh/gmsh.git


In [ ]:
import gmsh
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.interpolate import RegularGridInterpolator
import sys
from pyproj import Transformer

def read_elevation_from_file(hgt_file, SAMPLES):
    """load topography data from file"""
    with open(hgt_file, 'rb') as hgt_data:
        # Each data is 16bit signed integer(i2) - big endian(>)
        elevations = np.fromfile(hgt_data, np.dtype('>i2'), SAMPLES*SAMPLES).reshape((SAMPLES, SAMPLES))        
        return np.flipud(elevations)
    
def remove_anomalies(topo):    
    """These data contain occassional voids from a number of causes such as shadowing, 
     phase unwrapping anomalies, or other radar-specific causes. Voids are flagged with the value -32768.
     we replace them by the average of the surrounding values"""
    anomalies = np.where( topo == -32768 )
    for x, y in zip(anomalies[0], anomalies[1]):
        slice = topo[max(0, x-1):x+2, max(0,y-1):y+2] # assuming a 5x5 square
        topo[x,y] = np.mean([i for i in slice.flatten() if i > 0])  # threshold is 0

### Downloading topography 

In [ ]:
# Lower left corner of the selected area 
lat, lon = 34, -119
min_latitude= lat 
min_longitude=lon

# Download it from NASA earthdata https://search.earthdata.nasa.gov/search
# dataset: NASA Shuttle Radar Topography Mission Global 3 arc second V003

filename = 'N%dW%03d.hgt'%(int(min_latitude),int(abs(min_longitude)))

### Reading topography data

In [ ]:
# Samples of the original topographic data in each direction
samples = 1201    
topo = read_elevation_from_file(filename, samples)
remove_anomalies(topo)

# Generate dimension array
x_lon = np.linspace(int(min_longitude),int(min_longitude)+1,samples)
y_lat = np.linspace(int(min_latitude),int(min_latitude)+1,samples)

print("projecting the node coordinates")

# transverse mercator projection
sProj = "+proj=tmerc +datum=WGS84 +k=0.9996 +lon_0=-118.5150 +lat_0=34.3440 +axis=enu"
transformer = Transformer.from_crs("epsg:4326", sProj, always_xy=True)
transformer_inv = Transformer.from_crs(sProj, "epsg:4326", always_xy=True)

# Epicenter of 1994 Northridge earthquake: geographic coordinates
lat_sou = 34.3440
lon_sou = -118.5150
xyz_sou = transformer.transform(lon_sou,lat_sou)

x,y = transformer.transform(x_lon,y_lat)

# Plot topography
fig, ax = plt.subplots(1,2,figsize=(14,5))

# Lat/Lon 
im1 = ax[0].imshow(topo, interpolation='bilinear', cmap='terrain',aspect='auto',
                   origin='lower', extent=[x_lon[0], x_lon[-1], y_lat[0], y_lat[-1]],
                   vmax=2500, vmin=0)
ax[0].set_title('Lon/Lat',fontsize=14)
ax[0].set_xlabel('Longitude ($^{\circ}$)', fontsize=12)
ax[0].set_ylabel('Latitude ($^{\circ}$)', fontsize=12)
ax[0].scatter(lon_sou,lat_sou, s=200, marker='*', c='r',label='Epicenter')
ax[0].legend(loc=1,prop={"size":12})

# UTM domain
im2 = ax[1].imshow(topo, interpolation='bilinear', cmap='terrain',aspect='auto',
                   origin='lower', extent=[x[0], x[-1], y[0], y[-1]],
                    vmax=2500, vmin=0)
ax[1].scatter(xyz_sou[0],xyz_sou[1], s=200, marker='*', c='r',label='Epicenter')
ax[1].xaxis.major.formatter.set_powerlimits((-2,1))
ax[1].yaxis.major.formatter.set_powerlimits((-2,1))
ax[1].set_xlabel('X (m)', fontsize=12)
ax[1].set_ylabel('Y (m)', fontsize=12)
ax[1].set_title('UTM projection',fontsize=14)
ax[1].legend(loc=1,prop={"size":12})
fig.colorbar(im2,ax=ax[1],label='Elevation (m)', fraction=0.046, pad=0.025)

plt.show()

### Mesh generation

In [ ]:
# setup Model geometry dimension: 70 km by 70 km by 35 km

xmin = xyz_sou[0]-35000. # Unit (m)
xmax = xyz_sou[0]+35000. 

ymin = xyz_sou[1]-35000.
ymax = xyz_sou[1]+35000.

zmax = -35000.


# start to mesh using GMSH; Please specify element size lc and lc_surf as desired.

# Initialize Gmsh API
gmsh.initialize()
gmsh.model.add("Northridge")

## set Element size in meter
lc = 5e3 
lc_surf = 2.0e3

# Resample original topographic data: 1201 by 1201 to 101 by 101
# Create the terrain mesh with N by N data points:
N = 100

# Helper function to return a node tag given two indices i and j:
def tag(i, j):
    return (N + 1) * i + j + 1

# The x, y, z coordinates of all the nodes:
coords = []

# The tags of the corresponding nodes:
nodes = []

# The connectivities of the triangle elements (3 node tags per triangle) on the
# terrain surface:
triangles_connect = []

# The connectivities of the line elements on the 4 boundaries (2 node tags
# for each line element):
lines_connect = [[], [], [], []]

# The connectivities of the point elements on the 4 corners (1 node tag for each
# point element):
points_connect = [tag(0, 0), tag(N, 0), tag(N, N), tag(0, N)]

# Adding topography point by point
x_grid = np.linspace(xmin,xmax,N+1)
y_grid = np.linspace(ymin,ymax,N+1)
xx_grid, yy_grid = np.meshgrid(x_grid, y_grid)

# Interpolate grid topography
ftopo = RegularGridInterpolator((x_lon, y_lat), topo.T)
lon_lat_grid = transformer_inv.transform(xx_grid.flatten(),yy_grid.flatten())
topo_grid = ftopo(lon_lat_grid).reshape(N+1,N+1)

for i in range(N + 1):
    for j in range(N + 1):
        nodes.append(tag(i, j))
        coords.extend([x_grid[i],y_grid[j],topo_grid[j,i]]) 
        if i > 0 and j > 0:
            triangles_connect.extend([tag(i - 1, j - 1), tag(i, j - 1), tag(i - 1, j)]) 
            triangles_connect.extend([tag(i, j - 1), tag(i, j), tag(i - 1, j)])
        if (i == 0 or i == N) and j > 0:
            lines_connect[3 if i == 0 else 1].extend([tag(i, j - 1), tag(i, j)])
        if (j == 0 or j == N) and i > 0:
            lines_connect[0 if j == 0 else 2].extend([tag(i - 1, j), tag(i, j)])

# Create 4 discrete points for the 4 corners of the terrain surface:
for i in range(4):
    gmsh.model.addDiscreteEntity(0, i + 1)
gmsh.model.setCoordinates(1, xmin, ymin, coords[3 * tag(0, 0) - 1])
gmsh.model.setCoordinates(2, xmax, ymin, coords[3 * tag(N, 0) - 1])
gmsh.model.setCoordinates(3, xmax, ymax, coords[3 * tag(N, N) - 1])
gmsh.model.setCoordinates(4, xmin, ymax, coords[3 * tag(0, N) - 1])

# Create 4 discrete bounding curves, with their boundary points:
for i in range(4):
    gmsh.model.addDiscreteEntity(1, i + 1, [i + 1, i + 2 if i < 3 else 1])

# Create one discrete surface, with its bounding curves:
gmsh.model.addDiscreteEntity(2, 1, [1, 2, -3, -4])

# Add all the nodes on the surface:
gmsh.model.mesh.addNodes(2, 1, nodes, coords)
gmsh.model.addPhysicalGroup(2, [1], 101) # Free-surface boundary label

# Add point elements on the 4 points, line elements on the 4 curves, and triangle elements on the surface:
for i in range(4):
    # Type 15 for point elements:
    gmsh.model.mesh.addElementsByType(i + 1, 15, [], [points_connect[i]])
    # Type 1 for 2-node line elements:
    gmsh.model.mesh.addElementsByType(i + 1, 1, [], lines_connect[i])
# Type 2 for 3-node triangle elements:
gmsh.model.mesh.addElementsByType(1, 2, [], triangles_connect)

# Reclassify the nodes on the curves and the points 
gmsh.model.mesh.reclassifyNodes()

# Create a geometry for the discrete curves and surfaces, so that we can remesh them later on:
gmsh.model.mesh.createGeometry()

# Create other entities to form one volume below the terrain surface:
p1 = gmsh.model.geo.addPoint(xmin, ymin, zmax)
p2 = gmsh.model.geo.addPoint(xmax, ymin, zmax)
p3 = gmsh.model.geo.addPoint(xmax, ymax, zmax)
p4 = gmsh.model.geo.addPoint(xmin, ymax, zmax)
c1 = gmsh.model.geo.addLine(p1, p2)
c2 = gmsh.model.geo.addLine(p2, p3)
c3 = gmsh.model.geo.addLine(p3, p4)
c4 = gmsh.model.geo.addLine(p4, p1)
c10 = gmsh.model.geo.addLine(p1, 1)
c11 = gmsh.model.geo.addLine(p2, 2)
c12 = gmsh.model.geo.addLine(p3, 3)
c13 = gmsh.model.geo.addLine(p4, 4)
ll1 = gmsh.model.geo.addCurveLoop([c1, c2, c3, c4]) 
s1 = gmsh.model.geo.addPlaneSurface([ll1]) # bot
ll3 = gmsh.model.geo.addCurveLoop([c1, c11, -1, -c10]) # fro
s3 = gmsh.model.geo.addPlaneSurface([ll3]) # fro
ll4 = gmsh.model.geo.addCurveLoop([c2, c12, -2, -c11])
s4 = gmsh.model.geo.addPlaneSurface([ll4]) # rig
ll5 = gmsh.model.geo.addCurveLoop([c3, c13, 3, -c12])
s5 = gmsh.model.geo.addPlaneSurface([ll5]) # bac 
ll6 = gmsh.model.geo.addCurveLoop([c4, c10, 4, -c13])
s6 = gmsh.model.geo.addPlaneSurface([ll6]) # lef
gmsh.model.addPhysicalGroup(2, [s1, s3, s4, s5, s6], 105)  # Absorbing boundary label
sl1 = gmsh.model.geo.addSurfaceLoop([s1, s3, s4, s5, s6, 1])
v1 = gmsh.model.geo.addVolume([sl1])
gmsh.model.addPhysicalGroup(3, [v1], 1)
gmsh.model.geo.synchronize()


# setup key mesh parameters
gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [1,2,3,4])
gmsh.model.mesh.field.setNumbers(1, "FacesList", [1])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", lc_surf)#/ 20
gmsh.model.mesh.field.setNumber(2, "LcMax", lc)
gmsh.model.mesh.field.setNumber(2, "DistMin", 5000)
gmsh.model.mesh.field.setNumber(2, "DistMax", 20000)

# Use the minimum of all the fields as the background mesh field
gmsh.model.mesh.field.add("Min", 7)
gmsh.model.mesh.field.setNumbers(7, "FieldsList", [2])
gmsh.model.mesh.field.setAsBackgroundMesh(7)
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write('mesh_northridge.msh2') # type 2 Gmsh file  

# Alternatively, launch the GUI to see the results:
if '-nopopup' not in sys.argv:
     gmsh.fltk.run()

gmsh.finalize()

In [ ]:
# transform the gmsh mesh into the SeisSol mesh format using pumgen
!pumgen -s msh2 mesh_northridge.msh2

## Part 2 - SeisSol simulation

We now model seismic wave propagation emanated by many moment tensor point sources 
comprising a kinematic finite earthquake model. The earthquake source kinematics are described as a set of point sources, each assigned with a moment tensor and slip rate function, often derived from solving an inverse problem using seismic and/or geodetic data.

![](TabordaRoten2015.png)
*Example of a kinematic source model: (a) finite slip representation of the 1994 M 6.7 Northridge earthquake composed of 140 Â 140 subfaults (Modified after Graves and Pitarka 2010), (b) subfault model concept for a double-couple point source with independent geometry and source-time function, and (c) a typical slip-rate function in time. From [Taborda & Roten, 2015](https://link.springer.com/referenceworkentry/10.1007%2F978-3-642-36197-5_240-1)*


Our model describtion:
+ kinematic rupture model from [Hartzell et al., 1996](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/96JB01883) embedded in a 3D subsurface model
+ we use the SCEC Standard Rupture Format ([SRF](https://strike.scec.org/scecpedia/Standard_Rupture_Format))
+ source geometry: 20 x 25 km planar fault, 40° dipping 
+ we smooth and interpolate the kinematic model in space and time for SeisSol

![](SourceModel.png)

### Parameter file: parameters.par

The parameter files defines input/output paths, spatial and temporal discretisation, global simulation parameters, and more. 

**Task:** Inspect the parameter file and consult the [documentation](https://seissol.readthedocs.io/en/latest/parameter-file.html) to understand each line. 

### Specify initial parameter distributions using easi 

The main parameter file links to several files describing distributions of parameters using the [easi](https://seissol.readthedocs.io/en/latest/easi.html) library and ending in *.yaml* written in the YAML markup language. Easi is a library for the Easy Initialization of model parameters in three (or less) dimensional domains and in time. Easi offers the possibility to parameterize the simulation without having to recompile SeisSol. 

<img src="easi.png" width="40%">

*A shematic illustrating easi using different types of maps and filters.*


### Convert the source model into an efficient data format suitable for HPC 

The Standard Rupture Format (SRF) uses a geographic coordinate system, but our mesh is cartesian. Also, ASCII files are not efficient to read in a large number of point sources or a densely sampled source time function.
Thus we will convert the source model into SeisSol's own Netcdf Rupture Format ([NRF](https://seissol.readthedocs.io/en/latest/standard-rupture-format.html#how-to-use-rconv)) using our tool [rconv](https://seissol.readthedocs.io/en/latest/standard-rupture-format.html#how-to-use-rconv).


**Task: Transform the coordinate system of the kinematic source model on your laptop.**

We find a suitable projection for your region of interest at https://epsg.io/. 
Note, the “-m” option which specifies the projection with which the computational mesh was created.
“+axis=enu” means x=east, y=north, u=up.

In [ ]:
!rconv -i northridge_resampled.srf -o northridge_resampled.nrf -x visualization.xdmf -m "+proj=tmerc +datum=WGS84 +k=0.9996 +lon_0=-118.5150 +lat_0=34.3440 +axis=enu"

### Run SeisSol

In [ ]:
!OMP_NUM_THREADS=4 SeisSol_Release_dhsw_4_elastic parameters.par

### Use ParaView 

**Task: visualise the simulation's surface output using ParaView.**
